<a href="https://colab.research.google.com/github/Marcusleeleelee/FTEC4998-4999/blob/main/FTEC4998_4999.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Scikit-learn models and utilities
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# Scikit-learn utilities
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.utils import resample

# PyTorch for neural network models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Progress bar for loops
from tqdm import tqdm

# Plotting accuracy curves and other metrics
import matplotlib.pyplot as plt

# Others
import inspect
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay
from tabpfn import TabPFNClassifier
from matplotlib.colors import ListedColormap
import warnings; warnings.filterwarnings("ignore")

# Google Colab specific (if needed)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# Step 1: Utils - ok
def uni_list(input): return list(set(input))

In [27]:
class Dataset():
    def __init__(self, file_path):
        self.dataset = pd.read_feather(file_path)
        self.original = self.dataset.copy()
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None
        self.label = 'loan_condition_cat'
        self.min_max_columns = ['annual_inc', 'year']
        self.means = {}
        self.stds = {}
        self.mins = {}
        self.maxs = {}

    def show(self, rows=10):
        return self.dataset.head(rows)

    def basic_processing(self):
        temp_func_2 = lambda x: {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G': 7}[str(x)]
        columns_to_delete = [
            'id', 'issue_d', 'home_ownership_cat', 'income_category', 'income_cat', 'term_cat', 'application_type_cat',
            'purpose_cat', 'interest_payment_cat', 'loan_condition'
        ]
        self.dataset.drop(columns=columns_to_delete, inplace=True)
        self.dataset['grade'] = self.dataset['grade'].apply(temp_func_2)
        self.dataset['final_d'] = self.dataset['final_d'].apply(lambda x: str(x)[-4:]).apply(int)
        self.dataset['year'] = self.dataset['year'].apply(lambda x: str(x)[-4:]).apply(int)
        self.dataset = pd.get_dummies(self.dataset, columns=['home_ownership', 'term', 'application_type',
                                                             'purpose', 'interest_payments', 'region'], dtype=int)


    def train_test_split(self, test_size=0.2, random_state=42):
        X = self.dataset.drop(columns=[self.label])
        y = self.dataset[self.label]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        self.original_columns = X.columns

        # Sort by index
        self.X_train.sort_index(inplace=True)
        self.X_train.reset_index(drop=True, inplace=True)
        self.X_test.sort_index(inplace=True)
        self.X_test.reset_index(drop=True, inplace=True)
        self.y_train.sort_index(inplace=True)
        self.y_train.reset_index(drop=True, inplace=True)
        self.y_test.sort_index(inplace=True)
        self.y_test.reset_index(drop=True, inplace=True)

    def preprocessing_train(self):
        # Separate columns for Min-Max and Z-score normalization
        columns_to_normalize = self.min_max_columns
        columns_to_scale = [col for col in self.X_train.columns if col not in columns_to_normalize]

        # Z-score normalization
        for col in columns_to_scale:
            mean = np.mean(self.X_train[col])
            std = np.std(self.X_train[col])
            self.means[col] = mean
            self.stds[col] = std
            self.X_train[col] = (self.X_train[col] - mean) / std

        # Min-Max normalization
        for col in columns_to_normalize:
            min_val = np.min(self.X_train[col])
            max_val = np.max(self.X_train[col])
            self.mins[col] = min_val
            self.maxs[col] = max_val
            self.X_train[col] = (self.X_train[col] - min_val) / (max_val - min_val)

        # Perform PCA
        selected_columns = self.perform_pca(self.X_train, n_components=35)
        self.X_train = self.X_train[selected_columns]


    def perform_pca(self, data, n_components):
        # Center the data
        data_mean = np.mean(data, axis=0)
        centered_data = data - data_mean

        # Compute covariance matrix
        cov_matrix = np.cov(centered_data, rowvar=False)

        # Eigen decomposition
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Sort eigenvectors by eigenvalues in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        sorted_eigenvectors = eigenvectors[:, sorted_indices]

        # Select the top n_components
        selected_eigenvectors = sorted_eigenvectors[:, :n_components]

        # Identify important features
        feature_importance = np.abs(selected_eigenvectors).sum(axis=1)
        important_indices = np.argsort(feature_importance)[::-1][:n_components]

        # Return the original column names of these features
        important_features = [self.original_columns[i] for i in important_indices]

        return important_features

    def preprocessing_test(self):
        # Separate columns for Min-Max and Z-score normalization
        columns_to_normalize = self.min_max_columns
        columns_to_scale = [col for col in self.X_test.columns if col not in columns_to_normalize]

        # Apply Z-score normalization using training statistics
        for col in columns_to_scale:
            self.X_test[col] = (self.X_test[col] - self.means[col]) / self.stds[col]

        # Apply Min-Max normalization using training statistics
        for col in columns_to_normalize:
            self.X_test[col] = (self.X_test[col] - self.mins[col]) / (self.maxs[col] - self.mins[col])

        # Apply PCA using training components
        self.X_test = self.X_test[[i for i in self.X_train.columns.to_list()]]

In [28]:
# Calculating # ok
data = Dataset('/content/drive/My Drive/Colab Notebooks/FTEC4998_9/loan_final313_processed.feather')
data.basic_processing()
data.train_test_split()
data.preprocessing_train()
data.preprocessing_test()

In [51]:
# Data conversion # ok
train_x, train_y = data.X_train, data.y_train
test_x, test_y = data.X_test, data.y_test
counts = np.mean(train_y == 1) * 100
print(counts)
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)
# Ensure y_train is binary
assert set(train_y).issubset({0, 1}), "Target values must be 0 or 1 for binary classification."
# Move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

7.5910370853482805
(709903, 35) (709903,)
(177476, 35) (177476,)


In [52]:
print(train_x.shape); print(train_y.shape)
print(test_x.shape); print(test_y.shape)

(709903, 35)
(709903,)
(177476, 35)
(177476,)


In [53]:
# Train, predict, and accuracy functions
def train_model_pt(model): # ok
    model.train()
    for epoch in range(model.epochs):
        model.optimizer.zero_grad()
        outputs = model(model.train_x)
        loss = model.criterion(outputs, model.train_y)
        loss.backward()
        model.optimizer.step()
        if epoch % 5 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

def predict_model_pt(model, X):
    model.eval()
    with torch.no_grad():
        X = X.to(next(model.parameters()).device)
        outputs = model(X).squeeze()
        return (outputs > 0.5).float().cpu().numpy()  # Convert to numpy array

def calculate_accuracy_pt(model, X, y, pred=None):
    # Ensure X and y are on the same device as the model
    device = next(model.parameters()).device
    X = X.to(device)
    y = y.to(device)

    # Get predictions
    if pred is None: predictions = predict_model_pt(model, X)
    else: predictions = pred

    # Ensure predictions and labels are tensors and on the same device
    if not isinstance(predictions, torch.Tensor): predictions = torch.tensor(predictions).to(device)
    else: predictions = predictions.to(device)

    if not isinstance(y, torch.Tensor): y = torch.tensor(y).to(device)
    else: y = y.to(device)

    # Ensure predictions and labels are the same shape
    predictions = predictions.squeeze()
    y = y.squeeze()

    # Calculate accuracy
    correct = (predictions == y).sum().item()
    accuracy = correct / len(y)
    return accuracy

def df_to_tensor(x, y):
    assert isinstance(x, pd.DataFrame) and isinstance(y, pd.Series)
    return torch.tensor(x.to_numpy(), dtype=torch.float32).to(device), torch.tensor(y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
test_x, test_y = data.X_test, data.y_test
test_x_tensor, test_y_tensor = df_to_tensor(data.X_test, data.y_test)

In [59]:
# MLP model
class ANN(nn.Module): # Not yet finish the plot function
    def __init__(self, train_x, train_y, lr=0.01):
        super(ANN, self).__init__()
        self.train_y = torch.tensor(train_y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
        self.train_x = torch.tensor(train_x.to_numpy(), dtype=torch.float32).to(device)
        self.input_dim = self.train_x.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
        self._initialize_weights()
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 500

    def forward(self, x):
        return self.net(x)
    def _initialize_weights(self):
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

# Logistic Regression as a neural network
class LogisticRegressionModel(nn.Module): # Not yet finish the plot function
    def __init__(self, train_x, train_y, lr=0.001):
        super(LogisticRegressionModel, self).__init__()
        self.train_y = torch.tensor(train_y.values.ravel(), dtype=torch.float32).unsqueeze(1).to(device)
        self.train_x = torch.tensor(train_x.to_numpy(), dtype=torch.float32).to(device)
        self.input_dim = self.train_x.shape[1]
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 1),
            nn.Sigmoid()
        )
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.epochs = 1200

    def forward(self, x):
        return self.net(x)

# SVM
class SVMClassifier():
    def __init__(self, train_x, train_y, fraction=0.1, C=0.1, kernel='poly', degree=3, gamma='scale', n_estimators=6, max_samples=0.001, random_state=42):
        """
        Initialize the SVMClassifier with training data and optional parameters.

        Parameters:
        - train_x: Features for training.
        - train_y: Labels for training.
        - fraction: Fraction of data to sample (not used in this implementation).
        - C: Penalty parameter C of the error term for SVM.
        - kernel: Kernel type to be used in the SVM algorithm.
        - degree: Degree of the polynomial kernel function (if kernel='poly').
        - gamma: Kernel coefficient for 'rbf', 'poly', and 'sigmoid'.
        - n_estimators: The number of base estimators in the Bagging ensemble.
        - max_samples: The number of samples to draw from X to train each base estimator.
        - random_state: Controls the randomness of the bootstrapping of the samples.
        """
        self.train_x = train_x
        self.train_y = train_y
        self.C = C
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.random_state = random_state

        self.model = None
        self.accuracies = []

    def fit(self):
        """Fits the SVM model using Bagging with the specified parameters."""
        self.model = BaggingClassifier(
            estimator=SVC(C=self.C, kernel=self.kernel, degree=self.degree, gamma=self.gamma),
            n_estimators=self.n_estimators,
            max_samples=self.max_samples,
            random_state=self.random_state
        )
        self.model.fit(self.train_x, self.train_y)
        self._update_accuracy()

    def _update_accuracy(self):
        """Internal method to update and store the model's accuracy on the training set."""
        predictions = self.model.predict(self.train_x)
        accuracy = accuracy_score(self.train_y, predictions)
        self.accuracies.append(accuracy)

    def predict(self, X):
        """Predicts the labels for the given input data X."""
        if self.model is None:
            raise ValueError("Model has not been trained yet. Call `fit` before `predict`.")
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y, pred=None):
        """Calculates the accuracy of the model on the given data X and true labels y."""
        if pred is None:
            pred = self.predict(X)
        return accuracy_score(y, pred)

    def plot_training_curve(self):
        """Plots the training accuracy curve after fitting the model."""
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(self.accuracies) + 1), self.accuracies, marker='o', linestyle='-')
        plt.title('SVM Training Accuracy Curve')
        plt.xlabel('Iteration')
        plt.ylabel('Accuracy')
        plt.grid(True)
        plt.show()

# NB
class NaiveBayesClassifier():
    def __init__(self, train_x, train_y, priors=None, var_smoothing=1e-9):
        """
        Initialize the NaiveBayesClassifier with training data and model parameters.

        Parameters:
        - train_x: Features for training.
        - train_y: Labels for training.
        - priors: Prior probabilities of the classes. If specified, the priors are not adjusted according to the data.
        - var_smoothing: Portion of the largest variance of all features that is added to variances for stability.
        """
        self.train_x = train_x
        self.train_y = train_y
        self.accuracies = []

        self.model = GaussianNB(
            priors=priors,
            var_smoothing=var_smoothing
        )

    def fit(self):
        """Fit the Naive Bayes model on the training data and track training accuracy."""
        self.model.fit(self.train_x, self.train_y)
        accuracy = self._calculate_accuracy_internal(self.train_x, self.train_y)
        self.accuracies.append(accuracy)

    def _calculate_accuracy_internal(self, X, y):
        """Internal method to predict and calculate accuracy on the training data."""
        predictions = self.model.predict(X)
        return accuracy_score(y, predictions)

    def predict(self, X):
        """Predict the labels for the input features X."""
        if self.model is None:
            raise ValueError("Model has not been trained yet. Call `fit` before `predict`.")
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y, pred=None):
        """
        Calculate the accuracy of the model on the given data X and true labels y.

        Parameters:
        - X: Features for prediction.
        - y: True labels.
        - pred: Precomputed predictions (optional).

        Returns:
        - accuracy: Accuracy of the predictions.
        """
        predictions = self.model.predict(X).astype(float) if pred is None else pred
        return accuracy_score(y, predictions)

    def plot_training_curve(self):
        """Plot the training accuracy curve after fitting the model."""
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(self.accuracies) + 1), self.accuracies, marker='o', linestyle='-')
        plt.title('Naive Bayes Training Accuracy Curve')
        plt.xlabel('Iteration')
        plt.ylabel('Accuracy')
        plt.grid(True)
        plt.show()

# RF
class RandomForestModel():
    def __init__(self, train_x, train_y, n_estimators=10, max_depth=None, random_state=42, max_samples=0.05, warm_start=True):
        """
        Initialize the RandomForestModel with training data and hyperparameters.

        Parameters:
        - train_x: Features for training.
        - train_y: Labels for training.
        - n_estimators: The number of trees in the forest.
        - max_depth: The maximum depth of the tree.
        - random_state: Controls the randomness of the estimator.
        - max_samples: The number of samples to draw from X to train each base estimator.
        - warm_start: When set to True, reuse the solution of the previous call to fit.
        """
        self.train_x = train_x
        self.train_y = train_y
        self.accuracies = []

        self.model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state,
            max_samples=max_samples,
            warm_start=warm_start
        )

    def fit(self):
        """Fit the RandomForest model on the training data, tracking accuracy at each step."""
        for i in range(1, self.model.n_estimators + 1):
            self.model.n_estimators = i
            self.model.fit(self.train_x, self.train_y)
            accuracy = self._calculate_accuracy_internal(self.train_x, self.train_y)
            self.accuracies.append(accuracy)

    def _calculate_accuracy_internal(self, X, y):
        """Internal method to predict and calculate accuracy on the training data."""
        predictions = self.model.predict(X)
        return accuracy_score(y, predictions)

    def predict(self, X):
        """Predict the labels for the input features X."""
        if self.model is None:
            raise ValueError("Model has not been trained yet. Call `fit` before `predict`.")
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y, pred=None):
        """
        Calculate the accuracy of the model on the given data X and true labels y.

        Parameters:
        - X: Features for prediction.
        - y: True labels.
        - pred: Precomputed predictions (optional).

        Returns:
        - accuracy: Accuracy of the predictions.
        """
        predictions = self.model.predict(X).astype(float) if pred is None else pred
        return accuracy_score(y, predictions)

    def plot_training_curve(self):
        """Plot the training accuracy curve based on the number of trees in the forest."""
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(self.accuracies) + 1), self.accuracies, marker='o', linestyle='-')
        plt.title('Random Forest Training Accuracy Curve')
        plt.xlabel('Number of Trees')
        plt.ylabel('Accuracy')
        plt.grid(True)
        plt.show()
#XBG
class XGBoostClassifier():
    def __init__(self, train_x, train_y, n_estimators=100, learning_rate=0.1, max_depth=6,
                 subsample=0.8, colsample_bytree=0.8, gamma=0, reg_alpha=0, reg_lambda=1, eval_metric='error'):
        """
        Initialize the XGBoostClassifier with training data and hyperparameters.

        Parameters:
        - train_x: Features for training.
        - train_y: Labels for training.
        - n_estimators: Number of trees in the ensemble.
        - learning_rate: Step size shrinkage used to prevent overfitting.
        - max_depth: Maximum depth of a tree.
        - subsample: Subsample ratio of the training instances.
        - colsample_bytree: Subsample ratio of columns when constructing each tree.
        - gamma: Minimum loss reduction required to make a further partition.
        - reg_alpha: L1 regularization term on weights.
        - reg_lambda: L2 regularization term on weights.
        - eval_metric: Evaluation metric for cross-validation (default is 'error').
        """
        self.train_x = train_x
        self.train_y = train_y

        self.model = XGBClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            gamma=gamma,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            eval_metric=eval_metric
        )

    def fit(self):
        """Fit the XGBoost model on the training data."""
        self.model.fit(
            self.train_x, self.train_y,
            eval_set=[(self.train_x, self.train_y)],
            verbose=False
        )

    def predict(self, X):
        """Predict the labels for the input features X."""
        if self.model is None:
            raise ValueError("Model has not been trained yet. Call `fit` before `predict`.")
        return self.model.predict(X).astype(float)

    def calculate_accuracy(self, X, y, pred=None):
        """
        Calculate the accuracy of the model on the given data X and true labels y.

        Parameters:
        - X: Features for prediction.
        - y: True labels.
        - pred: Precomputed predictions (optional).

        Returns:
        - accuracy: Accuracy of the predictions.
        """
        predictions = self.model.predict(X).astype(float) if pred is None else pred
        return accuracy_score(y, predictions)

    def plot_training_curve(self):
        """Plot the training accuracy curve based on the model's evaluation results."""
        evals_result = self.model.evals_result()
        error_values = evals_result['validation_0']['error']
        accuracy_values = [1 - e for e in error_values]  # Convert error to accuracy
        epochs = len(accuracy_values)
        x_axis = range(epochs)

        plt.figure(figsize=(10, 6))
        plt.plot(x_axis, accuracy_values, marker='o', linestyle='-')
        plt.title('XGBoost Training Accuracy Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.grid(True)
        plt.show()

class TabPFNBagging():
    def __init__(self, train_x, train_y, n_estimators=10, sample_size=1000, device='cuda', N_ensemble_configurations=1):
        self.train_x = train_x
        self.train_y = train_y
        self.n_estimators = n_estimators
        self.sample_size = sample_size
        self.device = device
        self.N_ensemble_configurations = N_ensemble_configurations
        self.models = []
        self.X_sample, self.y_sample = None, None

    def fit(self):
        for i in range(self.n_estimators):
            print(f"Training estimator {i + 1}/{self.n_estimators}...")
            self.X_sample, self.y_sample = resample(self.train_x, self.train_y, n_samples=self.sample_size)
            model = TabPFNClassifier(device=self.device, N_ensemble_configurations=self.N_ensemble_configurations)
            model.fit(self.X_sample, self.y_sample, overwrite_warning=True)
            self.models.append(model)
        print("Bagging model training complete.")

    def predict(self, X, batch_size=9000):
        n_samples = X.shape[0]
        n_batches = int(np.ceil(n_samples / batch_size))

        # Initialize predictions array
        predictions = np.zeros((n_samples, len(self.models)))

        for i, model in enumerate(self.models):
            print(f"Predicting with model {i+1}/{self.n_estimators}...")
            for batch_idx in range(n_batches):
                start_idx = batch_idx * batch_size
                end_idx = min(start_idx + batch_size, n_samples)
                X_batch = X[start_idx:end_idx]

                y_eval, prob = model.predict(X_batch, return_winning_probability=True)
                predictions[start_idx:end_idx, i] = y_eval

        # Majority voting
        y_final = np.round(np.mean(predictions, axis=1)).astype(int)
        return y_final

    def calculate_accuracy(self, X, y, pred=None):
        y_pred = self.predict(X) if pred is None else pred
        accuracy = accuracy_score(y, y_pred)
        print(f"Bagging model accuracy: {accuracy:.4f}")
        return accuracy


In [ ]:
# @title temp1
tpfn = TabPFNBagging(train_x, train_y, n_estimators=5, sample_size=900, device=device)
tpfn.fit()
pred = tpfn.predict(test_x)
accuracy = tpfn.calculate_accuracy(test_x, test_y, pred)

In [57]:
class Adaboost():
    def __init__(self, classes_dict, train_x, train_y):
        self.classes_dict = classes_dict
        self.model_order = list(classes_dict.keys())
        self.train_x = train_x
        self.train_y = train_y
        self.trained_model = {}
        self.training_data_history = {'base': {'X': self.train_x, 'y': self.train_y}}
        self.current_weight = None
        self.weight_history = {}
        self.restart = False
        self.predictions = None

    def weight_init(self): self.current_weight = pd.Series(np.ones(len(self.train_y)) / len(self.train_y))

    def weight_calculate(self, predictions, labels):
        incorrect = predictions != labels.to_numpy()
        error_rate = self.current_weight[incorrect].sum()

        print('Error rate is:', error_rate)

        if error_rate > 0.5:
            self.weight_init()
            self.restart = True
            return

        alpha = 0.5 * np.log((1 - error_rate) / error_rate)

        # Update weights
        self.current_weight[incorrect] *= np.exp(alpha)
        self.current_weight[~incorrect] *= np.exp(-alpha)

        # Normalize weights
        self.current_weight /= self.current_weight.sum()

    def training(self):
        self.weight_init()
        for model in self.model_order:
            while True:
                self.restart = False
                self.train_x['weight'] = self.current_weight
                self.train_y = self.train_y.to_frame()
                self.train_y['weight'] = self.current_weight
                sampled_train_x = self.train_x.sample(n=len(self.train_x), replace=True, weights='weight', random_state=42)
                del self.train_x['weight']
                del sampled_train_x['weight']
                sampled_train_x.sort_index(inplace=True)
                sampled_train_x.reset_index(drop=True, inplace=True)
                sampled_train_y = self.train_y.sample(n=len(self.train_y), replace=True, weights='weight', random_state=42)
                del self.train_y['weight']
                self.train_y = self.train_y.iloc[:, 0]
                del sampled_train_y['weight']
                sampled_train_y.sort_index(inplace=True)
                sampled_train_y.reset_index(drop=True, inplace=True)
                sampled_train_y = sampled_train_y.iloc[:, 0]

                print("*" * 37)
                print(f'Training --------------------- {model}')
                current_model = self.classes_dict[model](sampled_train_x, sampled_train_y)
                methods = inspect.getmembers(current_model, predicate=inspect.ismethod)

                if 'fit' in [z for z, _ in methods]:
                    current_model.fit()
                    print('Finish training.\nStart predicting.')
                    current_prediction = current_model.predict(current_model.train_x)
                    train_accuracy = current_model.calculate_accuracy(current_model.train_x , current_model.train_y, current_prediction)
                else:
                    current_model.to(device)
                    train_model_pt(current_model)
                    print('Finish training.\nStart predicting.')
                    current_prediction = predict_model_pt(current_model, current_model.train_x)
                    train_accuracy = calculate_accuracy_pt(current_model, current_model.train_x, current_model.train_y, current_prediction)

                print(f'{model} training accuracy:', train_accuracy)
                self.weight_calculate(current_prediction, sampled_train_y)

                if not self.restart:
                    self.train_x, self.train_y = sampled_train_x, sampled_train_y
                    self.training_data_history[model] = {'X': sampled_train_x, 'y': sampled_train_y}
                    self.trained_model[model] = current_model
                    break
    def reorder(self, order_list): self.model_order = order_list
    def predict(self, X):
        # Ensure X is a pandas DataFrame
        assert isinstance(X, pd.DataFrame), "Input X should be a pandas DataFrame"
        # Collect predictions from each trained model
        model_predictions = {}
        for model_name, model in self.trained_model.items():
            methods = inspect.getmembers(model, predicate=inspect.ismethod)
            if 'predict' in [z for z, _ in methods]: preds = model.predict(X)
            else: preds = predict_model_pt(model, torch.tensor(X.to_numpy(), dtype=torch.float32).to(device))
            assert isinstance(preds, np.ndarray), f"Predictions from {model_name} should be a numpy array"
            if not np.array_equal(np.unique(preds), [0, 1]):
                print('Alert --- ', np.unique(preds))
                preds = np.where(preds > 0.5, 1, 0)
            model_predictions[model_name] = preds
        # Voting mechanism
        predictions = np.zeros(len(X))
        for i in range(len(X)):
            votes = {}
            for model_name, preds in model_predictions.items():
                pred = preds[i]
                if pred in votes: votes[pred] += 1
                else: votes[pred] = 1
            predictions[i] = max(votes, key=votes.get)
        assert isinstance(predictions, np.ndarray), "Final predictions should be a numpy array"

        self.predictions = predictions
        return predictions
    def calculate_accuracy(self, y):
        assert isinstance(self.predictions, np.ndarray), "Predictions should be a numpy array"
        assert isinstance(y, pd.Series), "y should be a pandas Series"
        assert len(self.predictions) == len(y), "Predictions and y should have the same length"
        accuracy = np.mean(self.predictions == y.to_numpy())
        return accuracy


In [58]:
adModel = Adaboost(classes_dict={"tabpfn":TabPFNBagging , "lr": LogisticRegressionModel, "svm": SVMClassifier, "ann": ANN, "nb": NaiveBayesClassifier, "rf": RandomForestModel, "xgb": XGBoostClassifier}, train_x=train_x, train_y=train_y)
adModel.reorder(['svm', 'xgb', 'ann', 'rf', 'tabpfn'])
adModel.training()
print("*" * 37)
print("Adaboost model prediction: ", adModel.predict(train_x))
print("Adaboost model training accuracy: ", adModel.calculate_accuracy(train_y))

*************************************
Training --------------------- svm
Finish training.
Start predicting.
svm training accuracy: 0.9298495709977278
Error rate is: 0.07015042900227215
*************************************
Training --------------------- xgb
Finish training.
Start predicting.
xgb training accuracy: 0.9018429278366199
Error rate is: 0.09108158984705605
*************************************
Training --------------------- ann
Epoch 0, Loss: 0.7069424390792847
Epoch 5, Loss: 0.6641299724578857
Epoch 10, Loss: 0.6223646998405457
Epoch 15, Loss: 0.6039136052131653
Epoch 20, Loss: 0.5909849405288696
Epoch 25, Loss: 0.5775768756866455
Epoch 30, Loss: 0.5656229257583618
Epoch 35, Loss: 0.5538395643234253
Epoch 40, Loss: 0.5453466176986694
Epoch 45, Loss: 0.5355695486068726
Finish training.
Start predicting.
ann training accuracy: 0.722111330702927
Error rate is: 0.27980500840928474
*************************************
Training --------------------- rf
Finish training.
Start pre

/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Predicting with model 2/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Predicting with model 3/10...


...

Predicting with model 9/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Predicting with model 10/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Adaboost model prediction:  [1. 0. 0. ... 0. 0. 0.]
Adaboost model training accuracy:  0.9284536056334457


In [60]:
print("Adaboost model prediction: ", adModel.predict(test_x))
print("Adaboost model training accuracy: ", adModel.calculate_accuracy(test_y))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Predicting with model 1/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Predicting with model 2/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

...

Predicting with model 9/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Predicting with model 10/10...


/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_inference):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/tabpfn/scripts/transformer_prediction_interface.py:530: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(en

Adaboost model prediction:  [0. 0. 1. ... 0. 1. 0.]
Adaboost model training accuracy:  0.9190763821587145
